<a href="https://colab.research.google.com/github/caddi-shayashi/data_analysis_hub/blob/master/notebooks/sales_data_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 CADDi 営業データ分析ツール（セールスチーム専用）

## 🎯 このツールについて

**日本語で質問するだけ**で、自動的にデータ分析ができる営業チーム専用ツールです。
プログラミング知識は一切不要です！

### ✅ できること
- 💬 日本語で質問 → AIが自動でデータを取得・分析
- 📈 結果を自動でグラフ化
- 📝 分析結果をわかりやすく要約
- 💾 分析履歴を保存

### 📌 使い方
1. **ステップ1～4のセル**を順番に実行（▶ボタンをクリック）
2. **質問を入力**して分析実行
3. 結果を確認！

---
## 📋 ステップ1: 初期設定

**👉 下のセルの ▶ ボタンをクリックしてください**

In [1]:
# 必要なツールをインストール（初回のみ実行されます）
import subprocess
import sys

print("⏳ 必要なツールをインストールしています...")
print("（初回は2-3分かかります）\n")

packages = [
    'google-cloud-bigquery',
    'google-cloud-bigquery-storage',
    'google-generativeai',
    'pandas',
    'plotly',
    'db-dtypes'
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✅ インストール完了！")
print("\n📌 次のステップに進んでください")

⏳ 必要なツールをインストールしています...
（初回は2-3分かかります）

✅ インストール完了！

📌 次のステップに進んでください


---
## 📋 ステップ2: Google Cloud接続

**👉 下のセルの ▶ ボタンをクリックしてください**

In [2]:
# Google Cloudに接続
print("⏳ Google Cloudに接続しています...\n")

try:
    from google.colab import auth
    auth.authenticate_user()
    print("✅ Google Cloud接続完了！")
    print("\n📌 次のステップに進んでください")
except:
    print("ℹ️ ローカル環境で実行中です")
    print("\n📌 次のステップに進んでください")

⏳ Google Cloudに接続しています...

✅ Google Cloud接続完了！

📌 次のステップに進んでください


---
## 📋 ステップ3: AI設定（Gemini API）

### 🔑 APIキーの設定方法

1. Colabの左側メニューから **🔑（鍵アイコン）** をクリック
2. **「シークレットを追加」** をクリック
3. 名前: `GEMINI_API_KEY`
4. 値: 提供されたAPIキーを貼り付け
5. 下のセルを実行

**👉 設定が完了したら、下のセルの ▶ ボタンをクリック**

In [3]:
# AI（Gemini）の設定
import google.generativeai as genai

print("⏳ AI設定中...\n")

try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("✅ AI設定完了！")
    print("\n📌 次のステップに進んでください")
except Exception as e:
    print("❌ APIキーが見つかりません")
    print("\n📝 設定方法:")
    print("1. 左側メニューの 🔑（鍵アイコン）をクリック")
    print("2. 'シークレットを追加'をクリック")
    print("3. 名前: GEMINI_API_KEY")
    print("4. 値: 提供されたAPIキーを貼り付け")
    print("5. このセルを再実行")

⏳ AI設定中...

✅ AI設定完了！

📌 次のステップに進んでください


---
## 📋 ステップ4: 分析システムの起動

**👉 下のセルの ▶ ボタンをクリックしてください**

In [4]:
# 分析システムを起動
print("⏳ 分析システムを起動しています...\n")

import os
import pandas as pd
import numpy as np
from datetime import datetime
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# プロジェクト設定
PROJECT_ID = 'esperanto-drawer-prod'
client = bigquery.Client(project=PROJECT_ID)

# AIモデル設定
model = genai.GenerativeModel('gemini-2.5-flash')

# 分析履歴を保存
analysis_history = []

print("✅ 分析システム起動完了！")
print("\n🚀 準備完了！下の分析セクションで質問を入力してください")

⏳ 分析システムを起動しています...

✅ 分析システム起動完了！

🚀 準備完了！下の分析セクションで質問を入力してください


---
# 🔍 データ分析を実行

## 💡 質問の例

### 売上分析
- 「今月の売上トップ10社を教えて」
- 「前年同期比での成長率は？」
- 「製品別の売上推移を見せて」

### 顧客分析
- 「新規顧客の獲得状況は？」
- 「大口顧客の契約状況を教えて」
- 「地域別の顧客分布は？」

### 商談分析
- 「今四半期の商談状況は？」
- 「成約率が高い営業担当者は誰？」
- 「リードタイムが長い案件を見せて」

---

## 📝 あなたの質問を入力

**👉 下のセルを実行して、質問を入力してください**

In [ ]:
# ===== 質問を入力 =====
question = "今月の売上トップ10社を教えて"  # ← ここを自由に変更してください！

# 分析を実行
print(f"🔍 質問: {question}")
print("=" * 60)
print("\n⏳ 分析中...\n")

def analyze_sales_data(question):
    """営業データを分析する関数"""

    # SQLを生成
    prompt = f"""
    以下の質問に答えるBigQuery SQLを生成してください：

    質問: {question}

    利用可能なテーブル:
    - esperanto-drawer-prod.dm_business_planning.salesforce_account_mart (顧客データ)
    - esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart (商談データ)

    注意:
    - 必ずLIMIT 100を付ける
    - シンプルで実行可能なSQLのみ返す
    - SQLは```sql と ```で囲む
    """

    try:
        # SQL生成
        response = model.generate_content(prompt)
        sql = response.text

        # SQL抽出
        if '```sql' in sql:
            sql = sql.split('```sql')[1].split('```')[0].strip()
        elif '```' in sql:
            sql = sql.split('```')[1].split('```')[0].strip()

        print("📝 生成されたSQL:")
        print(sql)
        print("\n" + "=" * 60 + "\n")

        # SQL実行
        print("⏳ データ取得中...\n")
        df = client.query(sql).to_dataframe()

        # 結果表示
        print(f"✅ {len(df)}件のデータを取得しました\n")
        print("📊 データ（上位10件）:")
        display(df.head(10))

        # グラフ作成
        if len(df) > 0:
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns

            if len(numeric_cols) > 0 and len(non_numeric_cols) > 0:
                fig = px.bar(
                    df.head(20),
                    x=non_numeric_cols[0],
                    y=numeric_cols[0],
                    title=f"分析結果: {question}",
                    template="plotly_white"
                )
                fig.update_layout(
                    xaxis_tickangle=-45,
                    height=500,
                    font=dict(size=12)
                )
                fig.show()

        # 要約生成
        print("\n" + "=" * 60)
        print("\n📝 分析結果の要約:")

        summary_prompt = f"""
        以下のデータ分析結果を営業チーム向けに3行で要約してください：

        質問: {question}
        データ件数: {len(df)}件

        主な発見を簡潔に説明してください。
        """

        summary = model.generate_content(summary_prompt)
        print(summary.text)

        # 履歴に追加
        analysis_history.append({
            'timestamp': datetime.now(),
            'question': question,
            'rows': len(df)
        })

        return df

    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")
        print("\n💡 ヒント:")
        print("- より具体的な質問にしてみてください")
        print("- 期間を指定してみてください（例：'2024年の'）")
        print("- 別の表現で質問してみてください")
        return None

# 分析実行
result_df = analyze_sales_data(question)

---
# 📚 よく使う分析テンプレート

以下のセルには、よく使う分析の質問がプリセットされています。
実行するだけで結果が得られます！

## 1️⃣ 月次売上分析

In [ ]:
# 月次売上分析
analyze_sales_data("2024年の月別売上推移を教えて")

## 2️⃣ 顧客ランキング

In [ ]:
# 顧客ランキング
analyze_sales_data("売上が大きい顧客トップ20を教えて")

## 3️⃣ 商談ステータス

In [ ]:
# 商談ステータス
analyze_sales_data("現在進行中の商談をステージ別に集計して")

## 4️⃣ 新規顧客分析

In [ ]:
# 新規顧客分析
analyze_sales_data("今年獲得した新規顧客の数と売上を月別に教えて")

## 5️⃣ 成約率分析

In [ ]:
# 成約率分析
analyze_sales_data("商談の成約率を営業担当者別に教えて")

---
# 📊 分析履歴の確認

これまでに実行した分析の履歴を確認できます。

In [ ]:
# 分析履歴を表示
if analysis_history:
    print("📚 分析履歴")
    print("=" * 60)

    history_df = pd.DataFrame(analysis_history)
    history_df['時刻'] = history_df['timestamp'].dt.strftime('%H:%M:%S')
    history_df = history_df[['時刻', 'question', 'rows']]
    history_df.columns = ['時刻', '質問', 'データ件数']

    display(history_df)
else:
    print("📭 まだ分析を実行していません")

---
# 💡 ヒントとトラブルシューティング

## ✅ 効果的な質問のコツ

### 良い質問の例
- ✅ 「2024年1月の売上トップ10社」（期間が明確）
- ✅ 「ENTセグメントの顧客数」（条件が明確）
- ✅ 「商談金額が1000万円以上の案件」（数値条件が明確）

### 改善が必要な質問の例
- ❌ 「売上は？」（曖昧すぎる）
- ❌ 「良い顧客」（「良い」の定義が不明）

## 🔧 エラーが出た場合

1. **「APIキーが見つかりません」**
   - ステップ3の手順に従ってAPIキーを設定

2. **「データが取得できません」**
   - 質問をより具体的にしてみる
   - 期間を限定してみる

3. **「タイムアウト」**
   - データ量を減らす（例：「上位10件」を追加）

## 📞 サポート

問題が解決しない場合は、以下にお問い合わせください：
- Slackチャンネル: #data-analysis-hub
- メール: data-support@caddi.jp

---
# 🎯 次のステップ

## このツールをマスターしたら...

### 📈 より高度な分析
- コホート分析（顧客の時系列分析）
- LTV（顧客生涯価値）予測
- チャーンリスク分析

### 🤝 チーム連携
- 分析結果をレポートとして共有
- 定期的なダッシュボード作成
- KPIモニタリングの自動化

### 📚 学習リソース
- [BigQuery基礎](https://cloud.google.com/bigquery/docs)
- [データ分析入門](https://www.coursera.org/learn/data-analysis-with-python)

---

**🎉 お疲れ様でした！**

このツールを使って、データドリブンな営業活動を実現しましょう！